The first step is to find and retrieve Jenkinsfile(s) from various open source repositories. We decided to use GitHub's REST API for our application. We are trying to access a jenkinsfile's raw content so that we can append it to one text file that contains all the jenkinsfiles' contents. This process will streamline parsing and generating results of analyses later on.

We are using a Jupyter Notebook from Anaconda with Python 3.6, we use the following packages to start off with our task.

In [3]:
import requests # Helps working with HTTP requests such as GitHub API.
import os # For performing tasks on shell.
import subprocess # For performing tasks on shell.

In [9]:
'''GitHub REST API at work with the Python REQUESTS package.

We ran a query that gives us all the files whose names match the string "jenkinsfiles", following proper convention.'''

repositories = requests.get('https://api.github.com/search/code?q=filename:jenkinsfile', auth = ('kshirsagarpratik', 'Chinku95'))

In [22]:
''' Iteratively retrieving the raw text content of all the jenkinsfiles returned by the API call and appending to a single textfile called "Jenkinsfile."'''

print('Attempting to retrieve jenkinsfile(s) from various GitHub repositories through it\'s REST API...')
try:
    for repo in repositories.json()['items']:
        print (repo['url'])
        jenkinsfile = requests.get(repo['url'], auth = ('kshirsagarpratik', 'Chinku95')) # Authentication for GitHub
        print(jenkinsfile)
        jenkinsfile_content = requests.get(jenkinsfile.json()['download_url'], auth = ('kshirsagarpratik', 'Chinku95'))
        file_pointer = open('Jenkinsfile.txt', 'a+')
        file_pointer.write(jenkinsfile_content.text)
        file_pointer.close()
except Exception as e:
    print('Error occured in retrieving jenkinsfile(s)')
    # LOG THIS INTO SOME FILE

Attempting to retrieve jenkinsfile(s) from various GitHub repositories through it's REST API
https://api.github.com/repositories/1319302/contents/Jenkinsfile?ref=380ab025880d831102d5cbbf5b429f1ff291272b
<Response [200]>
https://api.github.com/repositories/77328211/contents/Jenkinsfile?ref=c502ef25554e741ef72b4811c36f2b44873f1daf
<Response [200]>
https://api.github.com/repositories/103814838/contents/Jenkinsfile?ref=30f1c85438f4a19a0841bf87e11faf30cc519fb8
<Response [200]>
https://api.github.com/repositories/29193296/contents/Jenkinsfile?ref=8d9bc89b0c4ab9c23bdf1d617466b9be37ddde20
<Response [200]>
https://api.github.com/repositories/103943805/contents/Jenkinsfile?ref=09d77d643f726386931a8c758d39ad6cd5796362
<Response [200]>
https://api.github.com/repositories/103448225/contents/Jenkinsfile?ref=d45715e3bd756d2fb2748f8deb1b79871eb7aa3e
<Response [200]>
https://api.github.com/repositories/78342459/contents/Jenkinsfile?ref=fb59802ca4cf94d1f76fc6017f99f6a419b7a893
<Response [200]>
https://a

def gitCommit() {
    sh "git rev-parse HEAD > GIT_COMMIT"
    def gitCommit = readFile('GIT_COMMIT').trim()
    sh "rm -f GIT_COMMIT"
    return gitCommit
}

node {
    // Checkout source code from Git
    stage 'Checkout'
    checkout scm

    // Build Docker image
    stage 'Build'
    sh "docker build -t qcon2016sfmontyo/qcon:${gitCommit()} ."

    // Log in and push image to GitLab
    stage 'Publish'
    withCredentials(
        [[
            $class: 'UsernamePasswordMultiBinding',
            credentialsId: 'qcon2016sfmontyo',
            passwordVariable: 'DOCKERHUB_PASSWORD',
            usernameVariable: 'DOCKERHUB_USERNAME'
        ]]
    ) {
        sh "docker login -u ${env.DOCKERHUB_USERNAME} -p ${env.DOCKERHUB_PASSWORD} -e demo@mesosphere.com"
        sh "docker push mesosphere/qcon:${gitCommit()}"
    }
}
